# EMG信号处理快速开始

这个notebook将带你快速体验EMG信号处理的完整流程。

## 📚 学习目标

- 了解EMG信号的基本形态
- 学会加载和可视化EMG数据
- 体验信号滤波效果
- 提取简单的特征
- 完成一个手势识别任务

## ⏱️ 预计时间：30分钟

## 步骤1: 导入必要的库

In [ ]:
import sys
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt

# 添加项目路径
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

# 配置中文显示
plt.rcParams['font.sans-serif'] = ['SimHei', 'Microsoft YaHei']
plt.rcParams['axes.unicode_minus'] = False

print("✓ 库导入成功！")

## 步骤2: 生成模拟EMG信号

首先，让我们生成一些模拟的EMG信号来理解它的特征。

In [ ]:
# 生成模拟EMG信号
fs = 1000  # 采样率 1000Hz
duration = 5  # 持续时间 5秒
t = np.linspace(0, duration, int(duration * fs))

# 静息期：低幅度噪声
rest_signal = np.random.normal(0, 0.02, len(t))

# 收缩期：高幅度信号（1-4秒）
contraction_start = int(1 * fs)
contraction_end = int(4 * fs)

# 在收缩期添加多频率成分
for freq in range(60, 150, 20):
    rest_signal[contraction_start:contraction_end] += 0.3 * np.sin(
        2 * np.pi * freq * t[contraction_start:contraction_end]
    )

# 可视化
plt.figure(figsize=(14, 4))
plt.plot(t, rest_signal, linewidth=0.5)
plt.axvspan(0, 1, alpha=0.2, color='green', label='静息期')
plt.axvspan(1, 4, alpha=0.2, color='red', label='收缩期')
plt.axvspan(4, 5, alpha=0.2, color='green', label='放松期')
plt.xlabel('时间 (秒)')
plt.ylabel('幅度 (mV)')
plt.title('模拟EMG信号：静息 → 收缩 → 放松')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print("✓ EMG信号生成成功！")
print(f"  信号长度: {len(rest_signal)} 个采样点")
print(f"  采样率: {fs} Hz")
print(f"  时长: {duration} 秒")

## 步骤3: 信号滤波

EMG信号通常需要滤波来去除噪声。让我们应用一个带通滤波器。

In [ ]:
from code.week06_preprocessing.filters import EMGFilters

# 创建滤波器
filters = EMGFilters(fs=1000)

# 应用带通滤波器 (20-500 Hz)
filtered_signal = filters.bandpass_filter(rest_signal, lowcut=20, highcut=500)

# 对比原始信号和滤波后的信号
fig, axes = plt.subplots(2, 1, figsize=(14, 8))

# 原始信号
axes[0].plot(t, rest_signal, linewidth=0.5, color='blue')
axes[0].set_ylabel('幅度 (mV)')
axes[0].set_title('原始EMG信号')
axes[0].grid(True, alpha=0.3)

# 滤波后的信号
axes[1].plot(t, filtered_signal, linewidth=0.5, color='red')
axes[1].set_xlabel('时间 (秒)')
axes[1].set_ylabel('幅度 (mV)')
axes[1].set_title('滤波后的EMG信号 (20-500 Hz)')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("✓ 滤波完成！")

## 步骤4: 特征提取

从EMG信号中提取特征，用于后续的分类任务。

In [ ]:
from code.week07_feature_extraction.features import EMGFeatures

# 提取时域特征
time_features = EMGFeatures.extract_time_features(filtered_signal)

print("时域特征:")
for name, value in time_features.items():
    print(f"  {name:10s}: {value:.6f}")

# 提取频域特征
freq_features = EMGFeatures.extract_freq_features(filtered_signal, fs=1000)

print("\n频域特征:")
for name, value in freq_features.items():
    print(f"  {name:12s}: {value:.6f}")

print("\n✓ 特征提取完成！")

## 步骤5: 使用真实数据（可选）

如果你已经生成了样本数据，可以运行以下代码加载真实的EMG数据。

In [ ]:
try:
    from code.week05_data_processing.data_loading import EMGDataLoader
    
    # 加载数据
    loader = EMGDataLoader(data_dir='../data/sample/', fs=1000)
    loader.print_dataset_info()
    
    # 加载一个试验
    time, signals, label = loader.load_single_trial(
        '../data/sample/subject_01/fist/trial_001.csv'
    )
    
    # 可视化多通道信号
    fig, axes = plt.subplots(signals.shape[1], 1, figsize=(14, 8))
    for ch in range(signals.shape[1]):
        axes[ch].plot(time, signals[:, ch], linewidth=0.5)
        axes[ch].set_ylabel(f'通道{ch}')
        axes[ch].grid(True, alpha=0.3)
    axes[-1].set_xlabel('时间 (秒)')
    plt.suptitle(f'真实EMG信号 - 手势: {label}')
    plt.tight_layout()
    plt.show()
    
    print("✓ 真实数据加载成功！")
    
except FileNotFoundError:
    print("⚠️  未找到样本数据")
    print("请先运行: python tools/generate_sample_data.py")

## 🎉 恭喜！

你已经完成了EMG信号处理的快速入门！

### 你学到了什么：
- ✅ EMG信号的基本形态（随机性、突发性）
- ✅ 信号滤波的作用和效果
- ✅ 如何提取时域和频域特征
- ✅ 如何加载和可视化真实数据

### 下一步：
1. 查看 `01_信号可视化入门.ipynb` - 深入学习信号可视化
2. 查看 `02_滤波器设计实验.ipynb` - 实验不同的滤波器
3. 查看 `03_特征提取实践.ipynb` - 掌握所有18种特征
4. 运行 `examples/complete_pipeline.py` - 完整的手势识别流程

### 相关资源：
- [学习资源快速索引](../docs/学习资源快速索引.md)
- [EMG学习资源汇总](../docs/EMG学习资源汇总.md)
- [视频教程清单](../resources/videos/视频资源清单.md)